<a href="https://colab.research.google.com/github/Chegde8/LLM_projects/blob/main/text_summarization/dialogue_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

%pip install -U datasets==2.17.0
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet
%pip install \
    transformers==4.27.2 --quiet
%pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", 

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

# Load dataset

In [3]:
dataset_name = "Samsung/samsum"
dataset = load_dataset(dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for Samsung/samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Samsung/samsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.w

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [6]:
# View examples from dataset

index = 10
print(f"Dialogue:\n {dataset['train'][index]['dialogue']}")
print('-'.join('' for x in range(100)))
print(f"Summary: {dataset['train'][index]['summary']}")

Dialogue:
 Lucas: Hey! How was your day?
Demi: Hey there! 
Demi: It was pretty fine, actually, thank you!
Demi: I just got promoted! :D
Lucas: Whoa! Great news!
Lucas: Congratulations!
Lucas: Such a success has to be celebrated.
Demi: I agree! :D
Demi: Tonight at Death & Co.?
Lucas: Sure!
Lucas: See you there at 10pm?
Demi: Yeah! See you there! :D
---------------------------------------------------------------------------------------------------
Summary: Demi got promoted. She will celebrate that with Lucas at Death & Co at 10 pm.


# Load LLM model

In [36]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [37]:
# Tokenize text
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

# Case 1 - Generate summary without prompt engineering

In [41]:
example_indices = [0, 20]

for index in example_indices:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(model.generate(inputs["input_ids"], max_new_tokens=70,)[0], skip_special_tokens=True)

    print(f'INPUT PROMPT:\n{dialogue}')
    print('-'.join('' for x in range(100)))
    print(f'BASELINE GROUND TRUTH SUMMARY:\n{summary}')
    print('-'.join('' for x in range(100)))
    print(f'MODEL GENERATED SUMMARY:\n{output}\n')

INPUT PROMPT:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
---------------------------------------------------------------------------------------------------
BASELINE GROUND TRUTH SUMMARY:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
---------------------------------------------------------------------------------------------------
MODEL GENERATED SUMMARY:
Amanda can't find Betty's number. Amanda will ask Larry if he has Betty's number. Hannah doesn't know him well. Amanda texted Larry last time they were at the park together.

INPUT PROMPT:
Deirdre: Hi Beth, how are you love?
Beth: Hi Aunt

# Case 2 - Zero shot inference

In [40]:
example_indices = [0, 20]
for index in example_indices:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following dialogue:

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(model.generate(inputs["input_ids"], max_new_tokens=70,)[0], skip_special_tokens=True)

    print(f'BASELINE GROUND TRUTH SUMMARY:\n{summary}')
    print('-'.join('' for x in range(100)))
    print(f'MODEL GENERATED SUMMARY:\n{output}\n')

BASELINE GROUND TRUTH SUMMARY:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
---------------------------------------------------------------------------------------------------
MODEL GENERATED SUMMARY:
Amanda can't find Betty's number. Amanda will ask Larry if she can find it.

BASELINE GROUND TRUTH SUMMARY:
Beth wants to organize a girls weekend to celebrate her mother's 40th birthday. She also wants to work at Deidre's beauty salon. Deidre offers her a few hours on Saturdays as work experience. They set up for a meeting tomorrow.
---------------------------------------------------------------------------------------------------
MODEL GENERATED SUMMARY:
Beth's mother's 40th birthday is in 6 weeks. Deirdre is looking for Saturday girls. Beth wants to try a bit of work experience in the salon. Deirdre will give Beth a tenner per session. Beth will try the beauty therapy.



# Case 3 - One shot inference

In [42]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

    return prompt

In [45]:
# Generate prompt
example_indices = [400]
test_indices = [0, 20]

for ind in test_indices:
  one_shot_prompt = make_prompt(example_indices, ind)
  #print(one_shot_prompt)

  # Generate summary
  summary = dataset['test'][ind]['summary']

  inputs = tokenizer(one_shot_prompt, return_tensors='pt')
  output = tokenizer.decode(model.generate(inputs["input_ids"],max_new_tokens=70,)[0], skip_special_tokens=True)

  # print(dataset['test'][ind]['document'])
  # print('-'.join('' for x in range(100)))
  print(f'BASELINE GROUND TRUTH SUMMARY:\n{summary}\n')
  print('-'.join('' for x in range(100)))
  print(f'MODEL GENERATED SUMMARY:\n{output}\n')

BASELINE GROUND TRUTH SUMMARY:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

---------------------------------------------------------------------------------------------------
MODEL GENERATED SUMMARY:
Amanda can't find Betty's number. Amanda asks Larry to call her. Hannah doesn't know him well. Amanda prefers to text him.

BASELINE GROUND TRUTH SUMMARY:
Beth wants to organize a girls weekend to celebrate her mother's 40th birthday. She also wants to work at Deidre's beauty salon. Deidre offers her a few hours on Saturdays as work experience. They set up for a meeting tomorrow.

---------------------------------------------------------------------------------------------------
MODEL GENERATED SUMMARY:
Beth's mother's 40th birthday is coming up in 6 weeks. Deirdre is looking for Saturday girls. Beth wants to try a bit of work experience in the salon. Deirdre will give Beth a tenner per session. Beth will try the beauty therapy.



# Case 4 - Few shot inference

In [46]:
# Generate prompt
example_indices = [30, 80, 120]
test_indices = [0, 20]

for ind in test_indices:
  few_shot_prompt = make_prompt(example_indices, ind)


  # Generate summary
  summary = dataset['test'][ind]['summary']

  generation_config = GenerationConfig(max_new_tokens=70)

  inputs = tokenizer(few_shot_prompt, return_tensors='pt')
  output = tokenizer.decode(model.generate(inputs["input_ids"], generation_config=generation_config,)[0], skip_special_tokens=True)

  print(f'BASELINE GROUND TRUTH SUMMARY:\n{summary}\n')
  print('-'.join('' for x in range(100)))
  print(f'MODEL GENERATED SUMMARY:\n{output}\n')

BASELINE GROUND TRUTH SUMMARY:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

---------------------------------------------------------------------------------------------------
MODEL GENERATED SUMMARY:
Amanda can't find Betty's number. Amanda asks Larry to call her. Hannah doesn't know him well. Amanda texted him.

BASELINE GROUND TRUTH SUMMARY:
Beth wants to organize a girls weekend to celebrate her mother's 40th birthday. She also wants to work at Deidre's beauty salon. Deidre offers her a few hours on Saturdays as work experience. They set up for a meeting tomorrow.

---------------------------------------------------------------------------------------------------
MODEL GENERATED SUMMARY:
Beth's mother's 40th birthday is in 6 weeks. Kira will have to come back from Uni. Beth wants to try a bit of work experience in the salon. Deirdre is looking for Saturday girls. Beth will meet Maxine tomorrow after school. Beth will try the beauty therapy. D